In [8]:
import numpy as np
import xarray as xr
def increment_number(number_str):
    # 自增数字部分
    new_number = str(int(number_str) + 1).zfill(2)  # 保持两位数格式
    return new_number
num1='1'
# 示例用法
number_str = "00"
# # 打印原始数据的经纬度范围
# print("Longitude range:", data['lon'].min().values, "to", data['lon'].max().values)
# print("Latitude range:", data['lat'].min().values, "to", data['lat'].max().values)
# print(data['u'].coords)
# print(data['v'].coords)
# print(data['s'].coords)

In [9]:
def clip_nc_region(nc_file,num):
    # 打开 NetCDF 文件
    data = xr.open_dataset(nc_file)
    # 确定经纬度范围
    lon_range = np.linspace(112.0, 117.1, data['u'].shape[2])
    lat_range = np.linspace(12.9, 15.7, data['u'].shape[1])
    u = xr.DataArray(data['u'].values, dims=['lv', 'latu', 'lonu'], coords={'lonu': lon_range, 'latu': lat_range})
    
    # 创建 DataArray 对象并赋予坐标
    lon_range = np.linspace(112.0, 117.1, data['v'].shape[2])
    lat_range = np.linspace(12.9, 15.7, data['v'].shape[1])
    v = xr.DataArray(data['v'].values, dims=['lv', 'latv', 'lonv'], coords={'lonv': lon_range, 'latv': lat_range})
    
    # 确定目标经纬度范围
    target_lon = data['lon'].values
    target_lat = data['lat'].values
    
    # 进行插值
    regrid_u = u.interp(lonu=target_lon, latu=target_lat, method='linear')
    regrid_v = v.interp(lonv=target_lon, latv=target_lat, method='linear')
    
    # 截取 s, t, zeta 使用的 lon 和 lat
    # 指定要截取的经纬度范围
    lon_min, lon_max = 112, 117.1  # 经度范围
    lat_min, lat_max = 12.9, 15.7  # 纬度范围
    subset_t = data['t'].sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
    subset_s = data['s'].sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
    subset_zeta = data['zeta'].sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
    subset_u = regrid_u.sel(lonu=slice(lon_min, lon_max), latu=slice(lat_min, lat_max))
    subset_v = regrid_v.sel(lonv=slice(lon_min, lon_max), latv=slice(lat_min, lat_max))
    # 创建新的数据集
    subset = xr.Dataset({
        't': subset_t,
        's': subset_s,
        'zeta': subset_zeta,
        'u': subset_u,
        'v': subset_v
    })
    #保存为新的 NetCDF 文件
    subset.to_netcdf('E:/DataSet/redos/REDOS_1.0_1994/'+num1+'/subset_file_'+num+'.nc')

In [10]:
for _ in range(31):  # 模拟从 "01" 自增到 "20"
    number_str = increment_number(number_str)
    num2=number_str
    nc_file = 'E:/DataSet/redos/REDOS_1.0_1994/'+num1+'/REDOS_1.0_199401'+num2+'.nc/REDOS_1.0_199401'+num2+'.nc'
    # 打开 NetCDF 文件
    data = xr.open_dataset(nc_file)
    clip_nc_region(nc_file,number_str)
    # print(data['u'].coords)
    # print(data['v'].coords)
    # print("u dimensions:", data['u'].dims)
    # print("v dimensions:", data['v'].dims)
    # print("s dimensions:", data['s'].dims)

Coordinates:
    *empty*
Coordinates:
    *empty*
u dimensions: ('lv', 'latu', 'lonu')
v dimensions: ('lv', 'latv', 'lonv')
s dimensions: ('lv', 'lat', 'lon')


((24, 28, 52), (24, 28, 52), (28, 52), (24, 28, 52), (24, 28, 52))

In [76]:
import xarray as xr

# 打开保存的 NetCDF 文件
subset_data = xr.open_dataset('subset_file_3.nc')
t=subset_data['t']
s=subset_data['s']
zeta=subset_data['zeta']
u=subset_data['u']
v=subset_data['v']
# 查看 `t` 数据的前几个元素

[[        nan         nan         nan ... -0.0466703  -0.05348295
  -0.04565118]
 [        nan         nan         nan ... -0.10002735 -0.10397036
  -0.1124068 ]
 [        nan         nan         nan ... -0.07137974 -0.08938967
  -0.05337893]
 ...
 [        nan         nan         nan ... -0.00163597 -0.02371669
  -0.01573563]
 [        nan         nan         nan ... -0.0214408  -0.03850015
  -0.01503139]
 [        nan         nan         nan ... -0.01528002 -0.0093494
  -0.02374521]]


In [68]:
def tempfunc(nc_file,type,depth):
    daily_data = []
    #zeta为海表没有深度，所以永远为0
    if type=='zeta':
        temp_lv0 = nc_file.variables[type][ :, :]
    else:
        temp_lv0 = nc_file.variables[type][depth, :, :]
    #模拟20天的数据
    for i in range(30):
        daily_data.append(temp_lv0)
    day_lon_lat = np.array(daily_data)
    return day_lon_lat

In [69]:
def create_dataset(data, time_step):
    dataX = []
    for i in range(data.shape[0] - time_step + 1):
        dataX.append(data[i:i + time_step])
    return np.array(dataX)
def read_raw_data(vtype, depth, time_step,nc_file):
    #训练用的数据是第0层，也就是海表，原来那个是按照深度进行划分的，这个nc文件是按天数进行划分的，这里只有一天，所以shape[0]=1
    train_argo = tempfunc(nc_file,vtype,0)
    label_argo = tempfunc(nc_file,vtype,depth)
    width = train_argo.shape[2] #对应经度
    lenth = train_argo.shape[1] #对应纬度
    X = create_dataset(train_argo, time_step)
    X = X.reshape(X.shape[0],time_step,lenth,width,1)
    Y = label_argo[time_step-1 : label_argo.shape[0]] 
    Y =Y.reshape(Y.shape[0],lenth,width,1)
    #X 转置维度，变为 (样本数, 时间步长, 通道数, 纬度, 经度)。
    #Y 转置维度，变为 (样本数, 时间步长， 经度, 纬度)。
    X = X.transpose(0,1,4,2,3)
    Y = Y.transpose(0,3,1,2)
    return X, Y

In [70]:
#这几个数据格式一样，但是内容不一样，读的分别是不同的列
import netCDF4 as nc
file_path = './subset_file_3.nc'
file_obj = nc.Dataset(file_path)
train_sssa,_=read_raw_data('s',0,3,file_obj)
train_ssha,_ = read_raw_data('zeta',0,3,file_obj) #海面高度异常（Sea Level Anomaly）,他写的是sla，但是这里是zeta
train_sswu,_ = read_raw_data('u',0,3,file_obj)#U vwnd分量的风速（即沿经度方向的风速）,这里是u
train_sswv,_ = read_raw_data('v',0,3,file_obj)#V vwnd分量的风速（即沿纬度方向的风速），这里是v
train_argo, label_argo = read_raw_data('t', 1, 3,file_obj)#temp 代表温度数据,预测深度为1时的海温

In [71]:
train_sssa.shape,train_ssha.shape,train_sswu.shape,train_sswv.shape,label_argo.shape,train_argo.shape

((28, 3, 1, 28, 52),
 (28, 3, 1, 28, 52),
 (28, 3, 1, 28, 52),
 (28, 3, 1, 28, 52),
 (28, 1, 28, 52),
 (28, 3, 1, 28, 52))